## YoLov3 object detection on video

#### Importing Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### reading weights and configurations of YoLov3-tiny model

In [2]:
net = cv2.dnn.readNet("yolov3-tiny.weights", "yolov3.cfg")

#### reading set of classes from coco names file

In [3]:
classes = []

with open("coco.names", "r") as f:
    classes = f.read().splitlines()

#### reading video input

In [4]:
video = cv2.VideoCapture("walking.avi")

#### defining bounding boxes of object detected in video frames 

In [5]:
while (video.isOpened()):
    _, frame = video.read()
    height, width, _ = frame.shape
    
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416,416), (0,0,0), swapRB= True, crop= False)
    
    net.setInput(blob)
    
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)
    
    boxes = []
    class_ids = []
    confidences = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))

    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[i]
            cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y+20), font, 1, (255,0,255), 1)
            
        cv2.imshow('video', frame)
        if cv2.waitKey(1) & 0XFF == ord('e'):
            break
            
video.release()
cv2.destroyAllWindows()